In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import numpy as np
import numpy.linalg as la
from scipy.integrate import RK45

import sympy as sym

from table_maker import *
from functools import partial
from itertools import *
from math import ceil

def cos_bell(x, center=0, width=2*np.pi, height=1):
    return (np.cos((x-center)/width*2*np.pi)+1)/2*height * np.heaviside(x-center+width/2,0) * np.heaviside(-x+center+width/2,0)

In [2]:
import matplotlib
matplotlib.rcParams.update({'font.size': 24})

From [Kilpatrick and Bressloff 2010](https://doi.org/10.1016/j.physd.2009.06.003)
$$\begin{align*}
    \mu u_t &= -u + \int_{-\infty}^\infty w(x,x^\prime) q(x^\prime,t) f( u(x^\prime,t) - a(x^\prime,t)) \ dx^\prime \\
    q_t &= \frac{1 - q}{\alpha} - \beta q f(u - a) \\
    \epsilon a_t &= -a + \gamma f(u - a)
\end{align*}$$

Modified version - remove synaptic depression $q$
$$\begin{align*}
    \mu u_t &= -u + \int_{-\infty}^\infty w(x,x^\prime) f( u(x^\prime,t) - a(x^\prime,t)) \ dx^\prime \\
    \alpha a_t &= -a + \gamma f(u - a)
\end{align*}$$
(note that parameters have been relabeled)

# Traveling pulse solution

Beginning at (Kilpatrick & Bressloff 2010)[https://www.sciencedirect.com/science/article/abs/pii/S0167278909001833] equations 3.30 and 3.31. We will use the same notation they use (i.e. we will use the $\alpha$ and $\epsilon$ they use and substitute back to our notation later.

In [3]:
import collections.abc
def collect_eqn(expr, terms):
    if not isinstance(terms, collections.abc.Container):
        term = terms # isn't a list or tuple
        left, right = 0,0
        for my_expr in expr.args:
            has_term = False
            for my_term in sym.postorder_traversal(my_expr):
                if term == my_term:
                    has_term = True
                    break
            if has_term:
                left += my_expr
            else:
                right -= my_expr
        return sym.Eq(left, right, evaluate=False)
    else: # isn't a list or tuple
        left_total, right_total = 0, expr
        for term in terms:
            left, right = collect_eqn(right_total, term).args
            left_total += left
            right_total = -right
        return sym.Eq(left_total, right_total, evaluate=False)
    
def my_print(expr):
    global eviron_print_tex
    if eviron_print_tex:
        print('$$' + sym.latex(expr) + '$$')
    else:
        display(expr)
        
def batch_print(expr_list, punctuation='.'):
    global eviron_print_tex
    if eviron_print_tex:
        print_str = '$$\\begin{align*}\n'
        for expr in expr_list:
            print_str += '\t' + sym.latex(expr).replace('=','&=') + '\\\\\n'
        # remove last new-line and add ending puncuation
        print_str = print_str[:-3] + punctuation + '\n'
        print_str += '\\end{align*}$$'
        print(print_str)
    else:
        for expr in expr_list:
            display(expr)

In [5]:
# eviron_print_tex = True
eviron_print_tex = False

c, Δ, α, β, ϵ, γ, θ = sym.symbols('c \\Delta \\alpha \\beta \\epsilon \\gamma \\theta', real=True)

one_plus = 1+α*β
cα = c*α

K0 = (c*α + 1)/( 2*(c+1)*(cα + one_plus) )
K1 = 1/ (2*(c+1)*one_plus)
K2 = β*c*α**2 / ( 2 * (c+1) * one_plus * (c*α + one_plus) )

L0 = (2*c+1) / 2 / (c+1) / one_plus - γ
L1 = (c*α - 1) / ( 2 * (c-1) * (cα - one_plus) )
L2 = β*c**2*α**4 / one_plus / (cα**2 - one_plus**2) / (α - one_plus) - β*c*α**2 / 2 / (c+1) / one_plus / (cα + one_plus)
L3 = 1/one_plus * ( 1 + β*c**2*α**4 / (cα**2 - one_plus**2) / (α - one_plus) ) + (cα**2*(1+β) - one_plus) / (c**2 - 1) / (cα**2 - one_plus**2)

# symbolic values for printing purposes
symK0, symK1, symK2, symL0, symL1, symL2, symL3 = sym.symbols('K_0 K_1 K_2 L_0 L_1 L_2 L_3')

expr1 = -θ + symK0 - symK1*sym.exp(-Δ) - symK2*sym.exp(-one_plus*Δ/cα)*sym.exp(-Δ)
expr2 = -θ + symL0 + symL1*sym.exp(-Δ) + symL2*sym.exp(-one_plus*Δ/cα) - symL3*sym.exp(-Δ/c) + γ*sym.exp(-Δ/(c*ϵ))

sym_pairs = [[0, expr1], [0, expr2], [symK0, K0], [symK1, K1], [symK2, K2], [symL0, L0], [symL1, L1], [symL2, L2], [symL3, L3]]

print('We begin with')
batch_print([ sym.Eq(symvar, var) for symvar, var in sym_pairs ])
    
print('Then set $\\beta=0$ to obtain')

sub_var , sub_to = β, 0
K0 = sym.simplify(K0.subs(sub_var, sub_to))
K1 = K1.subs(sub_var, sub_to)
K2 = K2.subs(sub_var, sub_to)
L0 = L0.subs(sub_var, sub_to)
L1 = L1.subs(sub_var, sub_to)
L2 = L2.subs(sub_var, sub_to)
L3 = L3.subs(sub_var, sub_to)
expr1 = expr1.subs(sub_var, sub_to)
expr2 = expr2.subs(sub_var, sub_to)
sym_pairs = [[0, expr1], [0, expr2], [symK0, K0], [symK1, K1], [symK2, K2], [symL0, L0], [symL1, L1], [symL2, L2], [symL3, L3]]

batch_print([ sym.Eq(symvar, var) for symvar, var in sym_pairs ])
    
print('Substituting, we find')
for var, val in sym_pairs[2:]:
    expr1 = expr1.subs(var, val)
    expr2 = expr2.subs(var, val)
expr1 = sym.simplify(expr1 * (c+1))
expr2 = sym.simplify(expr2 * (c+1) * (c-1))
expr2 = sym.simplify(sym.expand(expr2))
sym_pairs = [[0, expr1], [0, expr2], [symK0, K0], [symK1, K1], [symK2, K2], [symL0, L0], [symL1, L1], [symL2, L2], [symL3, L3]]
batch_print([ sym.Eq(symvar, var) for symvar, var in sym_pairs[:2] ])

print('Simplifing, we arrive at the following system of equations')
c_sub = sym.solve(expr1, c)[0]
expr1 = -c + c_sub

Eceps = 1 - sym.exp(-Δ/(c*ϵ))
Ec = 1 - sym.exp(-Δ/c)
E = 1 - sym.exp(-Δ)
expr3 = c**2 * Ec - (c**2 - 1) * (γ* Eceps + θ) - (c+1) * E * sym.Rational(1,2)
assert expr3.expand() - expr2.expand() == 0
expr2 = expr3

batch_print([sym.Eq(c, expr1+c), sym.Eq(0, expr3)])

print('Making the substitution')
E = sym.symbols('E')
my_print(sym.Eq(E, sym.exp(-Δ)))
print('we then have')
expr3 = expr1.subs(sym.exp(-Δ), E)
assert expr3.subs(E, sym.exp(-Δ)) - expr1 == 0
expr1 = expr3
expr3 = expr2.subs(sym.exp(-Δ), E).subs(sym.exp(-Δ/c), E**(1/c)).subs(sym.exp(-Δ/(ϵ*c)), E**(1/(ϵ*c)))
assert expr3.subs(E, sym.exp(-Δ)) - expr2 == 0
expr2 = expr3.expand().collect(c)

# factor more
expr3 = (c**2-1) * (γ*(E**(1/(ϵ*c)) - 1) + 1 - E**(1/c) - θ) + 1 - E**(1/c) + (c+1)*(E-1)/2
assert (expr3 - expr2).expand() == 0
expr2 = expr3

sym_pairs = [[0, expr1], [0, expr2], [symK0, K0], [symK1, K1], [symK2, K2], [symL0, L0], [symL1, L1], [symL2, L2], [symL3, L3]]
batch_print([ sym.Eq(0, expr1), sym.Eq(0, expr2) ])

print('In our notation, we replace $\\epsilon \\to 1/\\alpha$ to obtain')
batch_print([ sym.Eq(E, sym.exp(-Δ)), sym.Eq(0, expr1), sym.Eq(0, expr2.subs(ϵ,1/α)) ])


We begin with


Eq(0, K_0 - K_1*exp(-\Delta) - K_2*exp(-\Delta)*exp(\Delta*(-\alpha*\beta - 1)/(\alpha*c)) - \theta)

Eq(0, L_0 + L_1*exp(-\Delta) + L_2*exp(\Delta*(-\alpha*\beta - 1)/(\alpha*c)) - L_3*exp(-\Delta/c) + \gamma*exp(-\Delta/(\epsilon*c)) - \theta)

Eq(K_0, (\alpha*c + 1)/((2*c + 2)*(\alpha*\beta + \alpha*c + 1)))

Eq(K_1, 1/((2*c + 2)*(\alpha*\beta + 1)))

...

Eq(0, -E**(1/c) + (E - 1)*(c + 1)/2 + (c**2 - 1)*(-E**(1/c) + \gamma*(E**(1/(\epsilon*c)) - 1) - \theta + 1) + 1)

In our notation, we replace $\epsilon \to 1/\alpha$ to obtain


Eq(E, exp(-\Delta))

Eq(0, -E/(2*\theta) - c - 1 + 1/(2*\theta))

Eq(0, -E**(1/c) + (E - 1)*(c + 1)/2 + (c**2 - 1)*(-E**(1/c) + \gamma*(E**(\alpha/c) - 1) - \theta + 1) + 1)